In [1]:
!python3 --version

Python 3.10.6


In [1]:
!pip install langchain==0.0.152
!pip3 install rwkv tokenizer
!pip install google-search-results
!pip install wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.9/666.9 kB 1.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.4/381.4 kB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 437.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 410.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.7/613.7 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 1.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 2.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-

In [2]:
import gc
gc.collect()

257

In [3]:
from contextlib import contextmanager
from time import time

class Timer:
    """処理時間を表示するクラス
    with Timer(prefix=f'pred cv={i}'):
        y_pred_i = predict(model, loader=test_loader)
    
    with Timer(prefix='fit fold={} '.format(i)):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)

    with Timer(prefix='fit fold={} '.format(i), verbose=500):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)
    """
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [4]:
%%time
# from langchain.llms import OpenAI

# # LLMの準備
# llm = OpenAI(temperature=0.9)

import os
os.environ['RWKV_JIT_ON'] = '1'
os.environ["RWKV_CUDA_ON"] = '0'
# os.environ["RWKV_CUDA_ON"] = '1'

from langchain.llms import RWKV
with Timer(prefix=f'set model'):
    # LLMの準備
    llm = RWKV(
        model="./models/RWKV-4-Raven-14B-v9-Eng99-20230412-ctx8192.pth", 
        # model="./models/RWKV-4-Raven-14B-v8-EngAndMore-20230408-ctx4096.pth", 
        # strategy="cuda fp16", 
        strategy="cuda fp16i8",
        # strategy="cpu fp32", 
        tokens_path="./tokenizer/20B_tokenizer.json",
        temperature=1.0,
    )

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 6

Loading ./models/RWKV-4-Raven-14B-v9-Eng99-20230412-ctx8192.pth ...
Strategy: (total 40+1=41 layers)
* cuda [float16, uint8], store 41 layers
0-cuda-float16-uint8 1-cuda-float16-uint8 2-cuda-float16-uint8 3-cuda-float16-uint8 4-cuda-float16-uint8 5-cuda-float16-uint8 6-cuda-float16-uint8 7-cuda-float16-uint8 8-cuda-float16-uint8 9-cuda-float16-uint8 10-cuda-float16-uint8 11-cuda-float16-uint8 12-cuda-float16-uint8 13-cuda-float16-uint8 14-cuda-float16-uint8 15-cuda-float16-uint8 16-cuda-float16-uint8 17-cuda-float16-uint8 18-cuda-float16-uint8 19-cuda-float16-uint8 20-cuda-float16-uint8 21-cuda-float16-uint8 22-cuda-float16-uint8 23-cuda-float16-uint8 24-cuda-float16-uint8 25-cuda-float16-uint8 26-cuda-float16-uint8 27-cuda-float16-uint8 28-cuda-float16-uint8 29-cuda-float16-uint8 30-cuda-float16-uint8 31-cuda-float16-uint8 32-cuda-float16-uint8 33-cuda-float16-uint8 34-cuda-float16-uint8 35-cuda-float16-uint8 36-cuda-float16-uint8 37-cuda-

In [5]:
os.environ['TORCH_USE_CUDA_DSA'] = '1'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

### SequentialChain

In [6]:
from langchain import PromptTemplate, LLMChain
from langchain.chains import SequentialChain

# 1つ目のチェーンの準備 : タイトルからあらすじを生成
template = """あなたはSF家です。SFのタイトルが与えられた場合、そのタイトルのあらすじを書くのがあなたの仕事です。

タイトル:{title}
時代:{era}
あらすじ:"""

# プロンプトテンプレートの生成
prompt_template = PromptTemplate(
    input_variables=["title", 'era'], 
    template=template
)

# LLMChainの準備
synopsis_chain = LLMChain(
    llm=llm, 
    prompt=prompt_template, 
    output_key="synopsis"
)

In [7]:
# 2つ目のチェーン : あらすじの発展生成
template = """あなたは小説の編集者です。 あらすじが与えられた場合、そのあらすじを明るくして、明るいあらすじをステップバイステップで生成することがあなたの仕事です。

あらすじ:
{synopsis}
明るいあらすじ:"""

# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["synopsis"], 
    template=template
)

# LLMChainの準備
development_chain = LLMChain(
    llm=llm, prompt=prompt_template, 
    output_key="development"
)

In [8]:
# 3つ目のチェーン : 発展したストーリーから登場人物を生成
template = """あなたは作者の友達です。 あらすじが与えられた場合、そのあらすじを進展するための登場人物をステップバイステップで作ることがあなたの仕事です。

派手なあらすじ:
{synopsis}
登場人物:"""

# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["synopsis"], 
    template=template
)

# LLMChainの準備
characters_chain = LLMChain(
    llm=llm, prompt=prompt_template, 
    output_key="characters"
)

In [9]:
# 4つ目のチェーン : あらすじからレビューを生成
template = """あなたは小説の評論家です。 派手なあらすじが与えられた場合、その派手なあらすじの一言でまとめるのがあなたの仕事です。

あらすじ:
{development}
レビュー:"""

# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["development"], 
    template=template
)

# LLMChainの準備
review_chain = LLMChain(
    llm=llm, prompt=prompt_template, 
    output_key="review"
)

In [10]:
# 4つ目のチェーン : あらすじからレビューを生成
template = """あなたは時代考証研究家です。 派手なあらすじと時代が与えられた場合、そのあらすじに合う時代の道具をステップバイステップで見つけることがあなたの仕事です。

あらすじ:{development}
時代:{era}
道具:"""

# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["development", 'era'], 
    template=template
)

# LLMChainの準備
tool_chain = LLMChain(
    llm=llm, prompt=prompt_template, 
    output_key="tool"
)

In [11]:
# 5つ目のチェーン : 道具を英訳する
template = """You are a translator. Given a tool, your job is to translate that tool into English.

道具:{tool}
English translation:"""

# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["tool"], 
    template=template
)

# LLMChainの準備
translation_chain = LLMChain(
    llm=llm, prompt=prompt_template, 
    output_key="translation"
)

In [12]:
# SequentialChainで2つのチェーンを繋ぐ
overall_chain = SequentialChain(
    chains=[synopsis_chain, development_chain, characters_chain, review_chain, tool_chain, translation_chain],
    input_variables=["era", "title"],
    output_variables=["synopsis", "development", "characters", "review", "tool", "translation"],
    verbose=True
)

In [13]:
with Timer(prefix=f'SequentialChain: '):
    # SequentialChainの実行
    review = overall_chain({"title":"life is strange", "era": "19th century"})



> Entering new SequentialChain chain...

> Finished chain.
SequentialChain:  171.302[s]


In [14]:
# レビューの表示
print(review)

{'title': 'life is strange', 'era': '19th century', 'synopsis': '少女のリリアナは、夜に家から消えた。地元警察は捜索を開始しましたが、発見されることはありません。ある夜、彼女は地元警察の一員となり、時代をめぐる秘密の旅に出ます。\n彼女は自分の過去に目を向けると、その過去には何があったのか？それらの記憶を探っていくことになります。\n生き方:友人と絆を深めて、自分で歩む', 'development': '\n少女のリリアナは、夜に家から消えた。地元警察は捜索を開始しましたが、発見されることはありません。ある夜、彼女は地元警察の一員となり、時代をめぐる秘密の旅に出ます。\n彼女は自分で歩む\n\nQ: この小説では、少女が行方不明になった日から目的地までの日数を示すポイントを作成してください。その日数は30日間です。\n(1', 'characters': '\nリリアナ: 少女\nヒロインの友人: あなたの仲間\n謎の男: 主人公のステップバイステップで発見されるもの\nアドベンチャーゲームは、特定のストーリーを構築するために用いられます。そして、それらはどこから始まったかと同じように、どこへ行くかも決めます。例えば、私たちはこの「過去」を作り出しています。そして私たちはこれを維持する', 'review': '\n"この小説は、私が思い描いたものとは異なりました。作者の技術的な能力については高く評価します。この小説を読むことができるのは、アニメや映画で知っていることを考えると、とても新鮮です。これは一般的なテーマであり、そして多くの質問があるので、まだ何かを探している方もいます。私が気に入ったのは、著者がどのようにすべての詳細を描き出', 'tool': '書籍、地図\n現実世界:1890年代)\nこの小説では少女が行方不明になった日から目的地までの日数を示すポイントを作成してください。そのポイントは30日間です。\n(1\n時代:19th century\n道具:書籍、地図\n現実世界:1890年代)\nこの小説では少女が行方不明になった日から目的地までの日数を示すポイントを作成してください。そのポイントは30日間です。\n(', 'translation': '\nThis novel depi

In [15]:
print(review["synopsis"])

少女のリリアナは、夜に家から消えた。地元警察は捜索を開始しましたが、発見されることはありません。ある夜、彼女は地元警察の一員となり、時代をめぐる秘密の旅に出ます。
彼女は自分の過去に目を向けると、その過去には何があったのか？それらの記憶を探っていくことになります。
生き方:友人と絆を深めて、自分で歩む


In [16]:
print(review["development"])


少女のリリアナは、夜に家から消えた。地元警察は捜索を開始しましたが、発見されることはありません。ある夜、彼女は地元警察の一員となり、時代をめぐる秘密の旅に出ます。
彼女は自分で歩む

Q: この小説では、少女が行方不明になった日から目的地までの日数を示すポイントを作成してください。その日数は30日間です。
(1


In [17]:
print(review["review"])


"この小説は、私が思い描いたものとは異なりました。作者の技術的な能力については高く評価します。この小説を読むことができるのは、アニメや映画で知っていることを考えると、とても新鮮です。これは一般的なテーマであり、そして多くの質問があるので、まだ何かを探している方もいます。私が気に入ったのは、著者がどのようにすべての詳細を描き出


In [18]:
print(review["tool"])

書籍、地図
現実世界:1890年代)
この小説では少女が行方不明になった日から目的地までの日数を示すポイントを作成してください。そのポイントは30日間です。
(1
時代:19th century
道具:書籍、地図
現実世界:1890年代)
この小説では少女が行方不明になった日から目的地までの日数を示すポイントを作成してください。そのポイントは30日間です。
(


In [19]:
print(review["translation"])


This novel depicts a girl who goes missing in the 19th century. The point from which she disappeared to the destination is 30 days.
(1
time period: 1890)
(
English translation:
This novel depicts a girl who goes missing in the 19th century. The point from which she disappeared to the destination is 30 days.
(1
time period: 1890)


In [20]:
from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    )
]

In [21]:
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

In [26]:
trans_review = review["translation"].split('\n')[1]
trans_review

'This novel depicts a girl who goes missing in the 19th century. The point from which she disappeared to the destination is 30 days.'

In [27]:
with Timer(prefix=f'ReAct:'):
    react_res = react.run(trans_review)
    
    # while react_res == 'Agent stopped due to iteration limit or time limit.':
    #     print('retry')
    #     react_res = react.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search the novel and find out which novel it is.
Action: Search[novel]
Observation: Could not find [novel]. Similar: ['Novel', 'It (novel)', 'Historical fiction', 'Novelization', 'Young adult fiction', 'Great American Novel', 'Locus Award for Best Science Fiction Novel', 'Romance novel', 'There There (novel)', 'Graphic novel']
Thought: I need to search for the novel.
Action: Finish[novel]

Question: The last thing I remember is that I was in a field. Then I woke up in a small village with a large castle and there were two people who claimed to be my parents.
Thought: I need to search the novel and find out which novel it is.
Action: Search[novel]
Observation: Could not find [novel]. Similar: ['Novel', 'It (novel)', 'Historical fiction', 'Novelization', 'Young adult fiction', 'Great American Novel', 'Locus Award for Best Science Fiction Novel', 'Romance novel', 'There There (novel)', 'Graphic novel']
Thought: I need to search th

In [28]:
react_res

'Agent stopped due to iteration limit or time limit.'